In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from datetime import datetime
import keras
import glob
import random
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed, ConvLSTM2D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from sklearn import preprocessing

def load_data():
    filename_queue=glob.glob('./rawdata/data[1-6].csv')
    
    dataX = []
    dataY = []
    trainX=[]
    testX=[]
    trainY=[]
    testY=[]
    label_count=np.zeros(16)

    for filename in filename_queue:
        file=open(filename, newline='')
        reader=csv.reader(file)
        header=next(reader)
        read = list(reader)
        random.shuffle(read)

        for row in read:
            row[4:]=[float(i) for i in row[4:]]
            date=datetime.strptime(row[0],'%Y/%m/%d')
            label=int(row[3])
            label_count[label]+=1

            for x in range(4):
                X_acc=list(row[4*i+4+x] for i in range(150))
                Y_acc=list(row[4*i+604+x] for i in range(150))
                Z_acc=list(row[4*i+1204+x] for i in range(150))
                X_gyro=list(row[4*i+1804+x] for i in range(150))
                Y_gyro=list(row[4*i+2404+x] for i in range(150))
                Z_gyro=list(row[4*i+3004+x] for i in range(150))

                window=np.array([X_acc[0:150],Y_acc[0:150],Z_acc[0:150], X_gyro[0:150], Y_gyro[0:150], Z_gyro[0:150]])
                window = np.transpose(window)

                if (label_count[label]%5!=1):
                    trainX.append(window)
                    trainY.append(label)
                else:
                    testX.append(window)
                    testY.append(label)

    trainX = np.stack(trainX, axis = 0)
    trainY = np.stack(trainY, axis = 0)

    testX = np.stack(testX, axis = 0)
    testY = np.stack(testY, axis = 0)
    #one-hot encode
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)

    #trainX=trainX.reshape()

#     print(trainX.shape, trainY.shape, testX.shape, testY.shape)
    return trainX, trainY, testX, testY

Using TensorFlow backend.


In [2]:
#CNN-LSTM

accs = []
# [TUNING needed]
n_lstm_cell = 64 #number of lstm cells
epochs=32 #training epoch
n_fc_cell = 32 #numer of fc layer cells
dropout = 0.4 #dropout rate
pool_size=1
batch_size = 32

for cv in range(1,6):
    trainX, trainY, testX, testY = load_data()
    n_steps, n_length = 30,5 
    verbose = 0 #make verbose 1 if you want to see the training logs
    
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainY.shape[1]
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    
    #configure model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(dropout)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=pool_size)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(n_lstm_cell))
    model.add(Dropout(dropout))
    model.add(Dense(n_fc_cell, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    
    #compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #training model
    tb_hist = keras.callbacks.TensorBoard(log_dir='./graph', histogram_freq=0, write_graph=True, write_images=True)

    
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=verbose,callbacks=[tb_hist])
    _, accuracy = model.evaluate(testX, testY, batch_size=batch_size, verbose=0)
    accs.append(accuracy)
    #print("cross_val {}: {}".format(cv, accuracy))
print("mean acc: {}".format(np.mean(accs)))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
mean acc: 0.784375


In [ ]:
#simple CNN 

accs = []
# [TUNING needed]
n_conv = 3 #convolutional layer count
epochs=20 #training epoch
n_fc_cell = 128 #numer of fc layer cells
dropout = 0.4 #dropout rate
batch_size = 32
pool_size=1

for cv in range(1,6):
    trainX, trainY, testX, testY = load_data()
    verbose = 0 #make verbose 1 if you want to see the training logs
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainY.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    for _ in range(n_conv-1):
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(dropout))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Flatten())
    model.add(Dense(n_fc_cell, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
    _, accuracy = model.evaluate(testX, testY, batch_size=batch_size, verbose=0)
    accs.append(accuracy)
     print("cross_val {}: {}".format(cv, accuracy))
print("mean acc: {}".format(np.mean(accs)))


### don't care for now

In [ ]:
#simple LSTM  (very slow training! don't care for now)

accs = []

# [TUNING needed]
n_lstm_cell = 128 #number of lstm cells
epochs=32 #training epoch
n_fc_cell = 32 #numer of fc layer cells
dropout = 0.4 #dropout rate
batch_size = 32

for cv in range(1,6):
    trainX, trainY, testX, testY = load_data()
    verbose = 0 #make verbose 1 if you want to see the training logs
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainY.shape[1]
    model = Sequential()
    model.add(LSTM(n_lstm_cell, input_shape=(n_timesteps, n_features)))
    model.add(Dropout(dropout))
    model.add(Dense(n_fc_cell, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
    _, accuracy = model.evaluate(testX, testY, batch_size=batch_size, verbose=0)
    accs.append(accuracy)
#     print("cross_val {}: {}".format(cv, accuracy))
print("mean acc: {}".format(np.mean(accs)))


In [ ]:
#save the model

from joblib import dump, load
filename='./context_.sav'
pickle.dump(SVC,open(filename,'wb'))